In [1]:
# count_food2.py

from collections import Counter

import spacy
from spacy.pipeline import EntityRuler
import pandas as pd

# 1. 口コミデータの読み込み
df = pd.read_csv('text/miyajima_sample.csv', encoding='utf-8',
                 parse_dates=['旅行の時期'])

nlp = spacy.load('ja_ginza')

In [3]:
# 2. 固有表現抽出のルールを追加
patterns = [
    {'label': 'Dish', 'pattern': [
        {'TEXT': 'あなご'},
        {'TEXT': {'REGEX': r'飯|めし'}},
    ]},
    {'label': 'Dish', 'pattern': '揚げもみじ'},
    {'label': 'Dish', 'pattern': 'ガンス'},
    {'label': 'Dish', 'pattern': [
        {'TEXT': 'にぎり'},
        {'TEXT': '天'}
    ]}
]


ruler = nlp.add_pipe("entity_ruler", before="ner")
ruler.add_patterns(patterns)

In [4]:
# 3. 料理名をリストに追加
food = []
for doc in nlp.pipe(df['口コミ']):
    # 料理名（Dishタグ）を抽出
    for ent in doc.ents:
        if ent.label_ == 'Dish':
            food.append(ent.text)

# 4. 料理名の頻度をカウント
counter = Counter(food)

In [5]:
# 5. 表記ゆれを解消するための用語辞書を用意
# 用語辞書のキーは代表語，値は表記ゆれの単語のリスト
name2variants = {'あなご飯': ['穴子飯', 'あなごめし', '穴子めし'],
                 '揚げもみじ': ['揚げ紅葉', '揚げ紅葉饅頭']}

# 用語辞書を，表記揺れをチェックしやすい形式に変換
variant2name = {}       # キーは表記ゆれの単語，値は代表語
for name, variants in name2variants.items():
    variant2name[name] = name
    for variant in variants:
        variant2name[variant] = name

In [6]:
# 6. 表記ゆれを解消して料理名の頻度をカウント
counter2 = Counter()
for word, count in counter.items():
    name = variant2name.get(word, word)
    counter2[name] += count

# 7. 料理名の頻度を表示
print('count word')
for word, count in counter2.most_common():
    print(f'{count:>5} {word}')

count word
   35 饅頭
   31 揚げもみじ
   17 あなご飯
    6 焼き牡蠣
    6 にぎり天
    6 ガンス
    6 穴子
    4 アイスクリーム
    4 コーヒー
    4 お茶
    3 牡蠣料理専門店
    3 スタバ
    3 もみじ饅頭
    2 サンドdeおにぎり
    2 飯
    2 チーズ
    2 揚げ
    2 ビール
    2 クリーム
    2 日本酒
    1 カキフライ
    1 牡蠣飯
    1 牡蠣料理
    1 カクテル
    1 ジュース
    1 おにぎり
    1 博多屋
    1 出来立て熱々
    1 コッペパン
    1 コーヒー豆
    1 弁当
    1 白湯
    1 軽食
    1 竹輪
    1 地ビール
    1 ステーキ
    1 ケーキ
    1 紅葉揚げ
    1 アイス
    1 じみ饅頭
    1 顔はめ
    1 穴子入り竹輪
    1 お土産選び
    1 スターバックス
    1 抹茶ミルクティー
    1 タピオカミルクティー
    1 お好み焼き
    1 揚げ紅葉うまい
    1 揚げ天
    1 牡蠣のパスタ
    1 カフェ
    1 海鮮丼
    1 カレーパン
    1 アンパンマン
    1 焼き立て
    1 スターバックスコーヒー
    1 ねぎたこ
    1 プリップリの焼き牡蠣
    1 焼きがき
    1 醤油マヨ
    1 牡蠣
    1 桐葉菓
    1 焼きがきのはやし
    1 焼き
    1 牡蠣の美味しさを知れ
    1 鹿せんべい


In [7]:
# 「男性」の口コミに出現する料理名の頻度をカウント

# 料理名をリストに追加
food = []
for doc in nlp.pipe(df[df['性別'] == '男性']['口コミ']):  # 女性の口コミに限定
    # 料理名（Dishタグ）を抽出
    for ent in doc.ents:
        if ent.label_ == 'Dish':
            food.append(ent.text)

# 料理名の頻度をカウント
counter = Counter(food)

# 表記ゆれを解消して料理名の頻度をカウント
counter2 = Counter()
for word, count in counter.items():
    name = variant2name.get(word, word)
    counter2[name] += count

# 料理名の頻度を表示
print('●　男性の場合')
print('count word')
for word, count in counter2.most_common():
    print(f'{count:>5} {word}')

●　男性の場合
count word
   25 饅頭
   15 揚げもみじ
   10 あなご飯
    6 穴子
    3 焼き牡蠣
    3 スタバ
    3 お茶
    3 もみじ饅頭
    2 チーズ
    2 牡蠣料理専門店
    2 日本酒
    1 白湯
    1 コーヒー
    1 ステーキ
    1 ケーキ
    1 紅葉揚げ
    1 じみ饅頭
    1 穴子入り竹輪
    1 スターバックス
    1 飯
    1 お好み焼き
    1 揚げ紅葉うまい
    1 海鮮丼
    1 にぎり天
    1 クリーム
    1 プリップリの焼き牡蠣
    1 焼きがき
    1 ビール
    1 揚げ
    1 醤油マヨ
    1 牡蠣
    1 桐葉菓
    1 焼きがきのはやし
    1 焼き
    1 牡蠣の美味しさを知れ
    1 鹿せんべい


In [8]:
# 「女性」の口コミに出現する料理名の頻度をカウント

# 料理名をリストに追加
food = []
for doc in nlp.pipe(df[df['性別'] == '女性']['口コミ']):  # 女性の口コミに限定
    # 料理名（Dishタグ）を抽出
    for ent in doc.ents:
        if ent.label_ == 'Dish':
            food.append(ent.text)

# 料理名の頻度をカウント
counter = Counter(food)

# 表記ゆれを解消して料理名の頻度をカウント
counter2 = Counter()
for word, count in counter.items():
    name = variant2name.get(word, word)
    counter2[name] += count

# 料理名の頻度を表示
print('●　女性の場合')
print('count word')
for word, count in counter2.most_common():
    print(f'{count:>5} {word}')

●　女性の場合
count word
   16 揚げもみじ
   10 饅頭
    7 あなご飯
    6 ガンス
    5 にぎり天
    4 アイスクリーム
    3 焼き牡蠣
    3 コーヒー
    2 サンドdeおにぎり
    1 牡蠣料理専門店
    1 カキフライ
    1 牡蠣飯
    1 牡蠣料理
    1 カクテル
    1 ジュース
    1 おにぎり
    1 博多屋
    1 出来立て熱々
    1 コッペパン
    1 コーヒー豆
    1 飯
    1 弁当
    1 軽食
    1 竹輪
    1 地ビール
    1 アイス
    1 顔はめ
    1 お土産選び
    1 抹茶ミルクティー
    1 タピオカミルクティー
    1 揚げ
    1 揚げ天
    1 ビール
    1 牡蠣のパスタ
    1 カフェ
    1 カレーパン
    1 お茶
    1 クリーム
    1 アンパンマン
    1 焼き立て
    1 スターバックスコーヒー
    1 ねぎたこ
